In [ ]:
# import settings and functions
%run ./../../imports.ipynb

%run ./../L_monte_settings.ipynb

## Analysis setup

In [ ]:
prefix = "c6_L_DD_"

# which executable?

exe = data_driven_diffusion_snes
sumanalys = "sumanalys.csv"
ana_name = "ana_square_mexi_mixed_order"


#### run analysis

In [ ]:
# redefine if analysis should be run
%run ./../load_setup.ipynb

In [ ]:
if run_analysis:
    !rm sumanalys.csv # making sure the previous analysis doesn't affect this one

    # loop over noise_lisgt and ana_name_list
    for (noise_in, ana_csv_name) in zip(noise_list, ana_name_list):
        !rm out*
        # !{create_csv_dataset} -output_file {csv_name}.csv -my_dummy_noise_q 0 -my_dummy_noise_k {noise_in} -my_dummy_k 1 -my_dummy_range_dp 9.0 -my_dummy_count {int(dummy_count)}

        core_name = "../dummy_"
        dataset_name = core_name+str(int(noise_in*100)) 

        # for i in range(realisations):
        !{exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {dataset_name}.csv -write_long_error_file -rand_ini {monte_control}

        !mv ./sumanalys.csv ./{ana_csv_name}.csv
        !mbconvert out_sigma_ele_{str(int(monte_num-1))}.h5m {noise_in*100}_out_sigma_{str(int(monte_num-1))}.vtk
        !mbconvert out_monte_ele_errors_{str(int(monte_num-1))}.h5m {noise_in*100}_out_monte_ele_errors_{str(int(monte_num-1))}.vtk
        !mbconvert out_before_perturb_{str(int(monte_num-1))}.h5m {noise_in*100}_out_before_perturb_{str(int(monte_num-1))}.vtk
        !mbconvert out_after_perturb_{str(int(monte_num-1))}.h5m {noise_in*100}_out_after_perturb_{str(int(monte_num-1))}.vtk
        # !mv ./sumanalys_monte_sigma.csv ./{ana_csv_name}_sigma.csv
        !cp standard_deviation_max.csv {noise_in*100}_standard_deviation_max.csv

    # rewrite sumanalys_monte_sigma.csv to only consist or the last number of rows as items in noise list
    !tail -n {len(noise_list)} sumanalys_monte_sigma.csv > sumanalys_monte_sigma.csv



In [ ]:
!cat ./sumanalys_monte_sigma.csv

### Plot results

In [ ]:
naming_std = ["T", "g_x", "g_y", "q_x", "q_y"]

# read the data from sumanalys_monte_sigma.csv file with naming defined by naming_std
data_std = pd.read_csv("sumanalys_monte_sigma.csv", names=naming_std, index_col=False, skiprows=0)
print(data_std)


In [ ]:
# plot the results from csv file wrt noise 
plt.figure()
for name in naming_std:
    plt.plot(noise_list, data_std[name], label=name)
plt.legend(loc='best')
plt.grid(True, ls=':')
plt.xlabel('Standard deviation of noise')
plt.ylabel('Standard deviation of error')




In [ ]:
# plot just 'T'
plt.figure()
plt.plot(noise_list, data_std['T'], label='T')
plt.legend(loc='best')
plt.grid(True, ls=':')
plt.xlabel('Standard deviation of noise')
plt.ylabel('Standard deviation of error')


In [ ]:
%run ./../L_plot_save_sigmas.ipynb

In [ ]:
%run ./../L_plot_save_T_Q.ipynb

In [ ]:
print("Done")